# 深層強化学習

このノートブックでは，シングルエージェントの深層強化学習の実験を行います．

まず，必要なパッケージをインストールします．

In [ ]:
!apt-get install ffmpeg freeglut3-dev xvfb  # For visualization
!pip install "stable-baselines3[extra]==2.0.0a4"
!pip install swig
!pip install gymnasium[box2d]

使用するパッケージをインポートします．
`gymnasium` は強化学習の環境を提供するライブラリです．
`stable_baselines3` は強化学習アルゴリズムを提供するライブラリです．

In [ ]:
import gymnasium as gym
import numpy as np
from stable_baselines3 import PPO
from stable_baselines3.ppo.policies import MlpPolicy
from stable_baselines3.common.evaluation import evaluate_policy

強化学習アルゴリズムを実行する前に，環境を確認しましょう．以下は，`gymnasium` が描画した環境を動画に記録し，それをノートブック上で再生するためのコードです．

In [ ]:
# Video recording/playing functions based on © 2019 Antonin RAFFIN
# https://github.com/araffin/rl-tutorial-jnrr19/blob/sb3/1_getting_started.ipynb
import os
import base64
from pathlib import Path
from IPython import display as ipythondisplay
from stable_baselines3.common.vec_env import VecVideoRecorder, DummyVecEnv

os.system("Xvfb :1 -screen 0 1024x768x24 &")
os.environ['DISPLAY'] = ':1'

def show_videos(video_path="", prefix=""):
    html = []
    for mp4 in Path(video_path).glob("{}*.mp4".format(prefix)):
        video_b64 = base64.b64encode(mp4.read_bytes())
        html.append(
            """<video alt="{}" autoplay
                    loop controls style="height: 400px;">
                    <source src="data:video/mp4;base64,{}" type="video/mp4" />
                </video>""".format(
                mp4, video_b64.decode("ascii")
            )
        )
    ipythondisplay.display(ipythondisplay.HTML(data="<br>".join(html)))

def record_video(env_id, model, video_length=500, prefix="", video_folder="videos/"):
    eval_env = DummyVecEnv([lambda: gym.make(env_id, render_mode="rgb_array")])
    # Start the video at step=0 and record 500 steps
    eval_env = VecVideoRecorder(
        eval_env,
        video_folder=video_folder,
        record_video_trigger=lambda step: step == 0,
        video_length=video_length,
        name_prefix=prefix,
    )

    obs = eval_env.reset()
    for _ in range(video_length):
        action, _ = model.predict(obs)
        obs, _, _, _ = eval_env.step(action)

    # Close the video recorder
    eval_env.close()

## 環境と強化学習モデルの定義

以下のセルで，環境と強化学習モデルを定義します．
`gymnasium` であらかじめ用意されている環境は，登録名を指定するだけで環境を作成することができます．以下では，`CartPole-v1` という環境を作成しています．他の環境も試してみましょう．登録されている環境については https://gymnasium.farama.org/environments/classic_control/ などを参照してください．

また，それと同時に強化学習モデルも定義しています．
ここでは，`PPO` という方策勾配法に基づく強化学習モデルを生成しています．
- 第一引数では，方策を表現するニューラルネットワークを指定しています．`MlpPolicy` は，あらかじめ `stable-baselines3` で定義されている方策で，多層パーセプトロンに基づく基本的なニューラルネットワークを使って方策を表現します．
- 環境が異なれば，観測や行動の空間が異なるため，それに合わせてニューラルネットワークの形を定義する必要があります．このため，`PPO` のコンストラクタの第二引数で環境 `env` を渡す必要があります．
- `learning_rate` や `batch_size`, `n_steps`, `n_epochs` はハイパーパラメータです．これらを変化させることで学習の効率が変化する場合があります．これ以外のハイパーパラメータについては https://stable-baselines3.readthedocs.io/en/master/modules/ppo.html を参照してください．

In [ ]:
env_name = "CartPole-v1"
# env_name = "LunarLander-v2"
# env_name = "FrozenLake-v1"

env = gym.make(env_name)

model = PPO(MlpPolicy, env, verbose=1,
            learning_rate=0.0003, # 学習率
            batch_size=64, # バッチサイズ
            n_steps=2048, # 訓練データバッファのサイズ
            n_epochs=10, # エポック数
            device="auto",
            tensorboard_log="./tb"
            )

以下のセルでは，環境の中でエージェントを行動させてみて，描画された環境を動画にして表示します．ここでは，エージェントの方策には，初期状態の（ほぼランダムに動く）強化学習モデルを使っていることになります．また，エピソードごとの平均報酬を計算して表示しています．

動画が不連続に変化しているように見える箇所があると思いますが，これはエピソードが終了して初期状態に戻っているためです．`CartPole-v1` の環境の場合，以下のいずれかの条件を満たすとエピソードが終了して，初期状態に戻ります．（ https://gymnasium.farama.org/environments/classic_control/cart_pole/#episode-end ）

1. Termination: Pole Angle is greater than ±12°
2. Termination: Cart Position is greater than ±2.4 (center of the cart reaches the edge of the display)
3. Truncation: Episode length is greater than 500 (200 for v0)

In [ ]:
# Show video and calculate mean reward before learning

record_video(env_name, model, video_length=500, prefix=f"ppo-{env_name}")
show_videos("videos", prefix=f"ppo-{env_name}")

mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=100, warn=False)
print(f"mean_reward: {mean_reward:.2f} +/- {std_reward:.2f}")

## 学習

以下のコードで，強化学習モデルの学習を行います．具体的には，以下の処理が，指定されたステップ数に達するまで実行されます．
- `collect_rollouts`: ハイパーパラメータ `n_steps` で指定したステップ数だけ，現在の方策を使って環境の中で行動します．この時に得られた観測や行動，報酬などのデータを，訓練データバッファに格納します．
- `train`: 訓練データバッファに格納されたデータを用いて，方策勾配法に基づいたニューラルネットワークの訓練を行います．

In [ ]:
# Learning
model.learn(total_timesteps=20000, tb_log_name="PPO")

以下のセルで，再び動画と平均報酬を表示しています．学習がうまくいって，適切な方策が得られた場合，より上手にタスクを遂行できていることが確認できると思います．

In [ ]:
# Show video and calculate mean reward after learning

record_video(env_name, model, video_length=500, prefix=f"ppo-{env_name}-trained")
show_videos("videos", prefix=f"ppo-{env_name}-trained")

mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=100)
print(f"mean_reward:{mean_reward:.2f} +/- {std_reward:.2f}")

以下のセルで，学習の過程のログを表示するTensorboardを開きます．
例えば，「ep_rew_mean」というログは，エピソードごとの平均報酬の推移を表しています．

In [ ]:
%load_ext tensorboard
%tensorboard --logdir=./tb

## 演習課題

- CartPole以外の環境を試してみなさい．
- PPOのハイパーパラメータを変更して，学習に与える影響を確認してみなさい．